In [2]:
from transformers import BartForConditionalGeneration, AutoTokenizer
bart = BartForConditionalGeneration.from_pretrained('hyunwoongko/kobart')
tokenizer = AutoTokenizer.from_pretrained('hyunwoongko/kobart')

In [3]:
texts = ['최근 들어 딥러닝', '안녕 친구야']
sample_ids = tokenizer(texts, padding=True, return_tensors='pt')['input_ids']
sample_ids

tensor([[19261, 14301,  1700, 10021, 10277,  9747],
        [11699,  9592, 15230, 11734,     3,     3]])

In [4]:
outs = bart.generate(
    sample_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=1,
    early_stopping=True,
)

In [31]:
import torch
import torch.nn.functional as F

def rerank(matrix, repeat_time, margin=0.):
    upper = matrix.unsqueeze(-1).repeat_interleave(repeat_time, dim=-1)
    lower = matrix.unsqueeze(1).repeat_interleave(repeat_time, dim=1)
    return F.relu((lower - upper + margin).triu(1))

In [32]:
batch_size = 8
num_cands = 16
# # embed_size = docs['input_ids'].size(-1)
# assert num_cands == cands['input_ids'].size(0) // batch_size

# doc_embeddings = self.encoder(**docs)[0][:, 0, :]
# gold_embeddings = self.encoder(**golds)[0][:, 0, :]
# sims_doc_gold = torch.cosine_similarity(doc_embeddings, gold_embeddings, dim=-1)

# cand_embeddings = self.encoder(**cands)[0][:, 0, :].view(batch_size, num_cands, -1)
# doc_embeddings = doc_embeddings.unsqueeze(1).repeat_interleave(num_cands, dim=1)
# sims_doc_cand = torch.cosine_similarity(doc_embeddings, cand_embeddings, dim=-1)

# scores_gold = sims_doc_cand - sims_doc_gold.unsqueeze(1).repeat_interleave(num_cands, dim=1)
# scores_gold = F.relu(scores_gold).sum(1)

ranks = (
    torch.arange(num_cands)
    .unsqueeze(0)
    .repeat_interleave(batch_size, dim=0)
)
rerank(ranks, num_cands)

tensor([[[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.]

In [33]:
import pandas as pd
pd.DataFrame({'ab': [[1,2,3],[1,2,3]]})

,ab
0,"[1, 2, 3]"
1,"[1, 2, 3]"


In [34]:
from datasets import load_metric
rouge = load_metric('rouge')

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [47]:
predictions = ['abc ccc', 'edf']
references = ['abc ccc', 'edf']
# ['최근 들어 딥러닝은 음성, 시각, 자연어, 로봇, 게임 등 인공지능 여러 분야의', '환경을 근본적으로 변화시키고 있다.']
# references = ['최근 들어 딥러닝은 음성, 시각, 자연어, 로봇, 게임 등 인공지능 여러 분야의', '환경을 근본적으로 변화시키고 있다.']
scores = rouge.compute(predictions=predictions, references=references[::-1])

In [48]:
scores['rouge1'].mid.fmeasure

0.0

In [1]:
valid_path = '/Volumes/GoogleDrive/My Drive/gassum/data/valid_w_cands.csv'

In [2]:
import pandas as pd

In [3]:
valid = pd.read_csv(valid_path)

In [5]:
valid = valid.iloc[:, 2:]

In [15]:
eval(valid['candidates'][1])

['바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하는 등 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.',
 '바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하는 등 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있는 것으로 알려졌다.',
 '바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하여 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.',
 '바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하는 등, 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.']

In [16]:
train_path = '/Volumes/GoogleDrive/My Drive/gassum/data/train_w_cands.csv'

In [17]:
train = pd.read_csv(train_path)

In [18]:
len(train)

271093

In [21]:
train

,id,title,text,summary,publish_date,size,char_count,category,media_type,media_sub_type,media_name,candidates
0,290741778,"논 타작물 재배, 2월 말까지 신청하세요",ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,2018-01-05 18:54:55,small,927,종합,online,지역지,광양신문,['전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정...
1,290741792,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,2018-01-05 18:53:20,small,764,종합,online,지역지,광양신문,"[""여수시는'낮에는 색채, 밤에는 빛'을 주제로 지난해 초 착공에 들어간 컬러빌리지..."
2,290741793,“새해 정기 받고 올해는 반드시 일내자!”,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,2018-01-05 18:52:15,medium,1066,스포츠,online,지역지,광양신문,['전남드래곤즈 구성원 모두가 한마음 한 뜻으로 구단 목표 달성을 위해 정진하자는 ...
3,290741794,"농업인 역량 강화, 새해 실용교육 실시","11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...",2018-01-05 18:50:17,small,746,정치,online,지역지,광양신문,['광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 10...
4,290741797,타이완 크루즈관광객 4천여명‘전남’온다,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,2018-01-05 18:52:36,small,978,종합,online,지역지,광양신문,"[""전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급'아쿠아리우스'..."
...,...,...,...,...,...,...,...,...,...,...,...,...
271088,340626140,미국서 날아온 비보...조기 걸린 대한항공,"[ 노유정 기자 ] 조양호 한진그룹 회장의 갑작스런 타계 소식이 전해진 8일 오전,...",8일 미국에서 들려온 조양호 한진그룹 회장의 사망 소식에 서울 공항동 대한항공 본사...,2019-04-08 17:41:00,medium,1143,경제,online,경제지,한국경제,"['조양호 한진그룹 회장의 갑작스런 타계 소식이 전해진 8일 오전, 서울 공항동 대..."
271089,340626209,"""항공·물류산업 세계적 반열 올려놓은 분""",] 조양호 한진그룹 회장이 타계하자 재계는 일제히 추모성명을 냈다. 전국경제인연합회...,조양호 한진그룹 회장이 타계하고 8일 전국경제인연합회는 논평을 통해 한국 항공과 물...,2019-04-08 17:42:00,small,849,경제,online,경제지,한국경제,"['조양호 한진그룹 회장이 타계하자 전국경제인연합회는 8일 논평에서""한국 항공·물류..."
271090,340626213,趙회장 지분 3500억대...할증세율까지 적용하면 상속세 2000억 웃돌듯,[ 김익환 기자 ] 조양호 한진그룹 회장이 타계하면서 그의 재산을 물려받을 장남 조...,"고 조 회장의 한진칼, 한진 등 상장회사의 주식가격 총합은 약 3560억 정도 되며...",2019-04-08 17:42:00,medium,1145,경제,online,경제지,한국경제,['8일 한국거래소에 따르면 조양호 한진그룹 회장이 타계하면서 그의 재산을 물려받을...
271091,340626365,"한진그룹, 사장단 중심 비상경영체제로...조원태 '3세 경영' 본격화",[ 김보형 기자 ] 조양호 한진그룹 회장이 8일 타계하면서 그의 장남인 조원태 대한...,"지난 8일 조양호 한진그룹 회장이 별세하면서 한진그룹은 비상경영체제에 돌입했고, 그...",2019-04-08 17:42:00,large,1899,경제,online,경제지,한국경제,"[""조양호 한진그룹 회장이 8일 타계하면서 그의 장남인 조원태 대한항공 사장(44)..."


In [20]:
train = train.iloc[:, 2:]

In [40]:
train.to_csv(train_path, index=False)

In [36]:
train = train.loc[train['summary'].map(lambda x: isinstance(x, str))]

In [39]:
valid.loc[~valid['summary'].map(lambda x: isinstance(x, str))]

,id,title,text,summary,publish_date,size,char_count,category,media_type,media_sub_type,media_name,candidates
